### Part 1.8 - Extract_data_40000_movies

#### Import the libraries

In [ ]:
# For cleaning and preparing the dataset
# -> dataframe manipulation
# -> text manipulation
# -> Web Scrapping

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook
# Example of Use: tqdm_notebook(examples, desc="Converting examples to features")

#### Eigtht 5000 movies (40,000)

Extract data about:

* field 1: Plot Summary
* field 2: Actors
* field 3: Directors
* field 4: IMDB rating
* field 5: Plot Synopsis
* field 6: Reviews

In [ ]:
"""
Open the movie_content urls for the eigtht 5000 movies and create the souplist of those URLs
"""
content_eight = []

with open(os.path.join(os.getcwd(), 'movie_content_url\\data_eight_13012020.pkl'), 'rb') as f:
    
    content_eight = pickle.load(f)

print("Number of URLs: {}".format(len(content_eight)))

content_souplist_eight = []

for i in tqdm_notebook(content_eight):
    
    content_souplist_eight.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_eight)))

In [ ]:
"""
Open the synopsis urls of the eigtht 5000 movies and create the souplist of those URLs
"""
synopsis_eight = []

with open(os.path.join(os.getcwd(), 'synopsis_url\\synopsis_eight_14012020.pkl'), 'rb') as f:
    
    synopsis_eight = pickle.load(f)

print("Number of URLs: {}".format(len(synopsis_eight)))

synopsis_souplist_eight = []

for i in tqdm_notebook(synopsis_eight):
    
    synopsis_souplist_eight.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(synopsis_souplist_eight)))

In [ ]:
"""
Open the reviews urls of the eigtht 5000 movies and create the souplist of those URLs
"""
review_eight = []

with open(os.path.join(os.getcwd(), 'reviews_url\\review_eight_15012020.pkl'), 'rb') as f:
    
    review_eight = pickle.load(f)

print("Number of URLs: {}".format(len(review_eight)))

review_souplist_eight = []

for i in tqdm_notebook(review_eight):
    
    review_souplist_eight.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_eight)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eight)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_eight)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eight)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name if len(item)!=0]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_eight)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract plot synopsis

Having extracted the synopsis content for each movie, we noticed that roughly more than 50% percent of the movies have a synopsis text.
Thus, in order not to erase almost half of the first 5,000 we will not further proceed in keeping the synopsis text
"""
synopsis_step_one = []
synopsis_step_two = []
synopsis_step_three = []

[synopsis_step_one.append(i.find_all('ul', {'class':'ipl-zebra-list', 'id':'plot-synopsis-content'})) for i in tqdm_notebook(synopsis_souplist_eight)]

[[synopsis_step_two.append(j.find_all('li', {'class':'ipl-zebra-list__item'})) for j in i] for i in synopsis_step_one]

[[synopsis_step_three.append(j.text.strip(' ').replace('\n', '').replace('\\', '')) for j in i] for i in synopsis_step_two]

index_to_remove_no_synopsis = [i for i,x in enumerate(synopsis_step_one) if not x]

print("Length of Synopsis list: {}".format(len(synopsis_step_three)))
print("Length of the list with Movies that don't have synopsis: {}".format(len(index_to_remove_no_synopsis)))
if len(index_to_remove_no_synopsis) == 0:
    print("None of the movies miss a synopsis")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_synopsis))

In [ ]:
"""
Print the movies with no synopsis text.
Result: We observed that many movies missed a synopsis text. Thus, we decided to not use synopsis for furhter analysis (although will be extracted)
"""
print([i for i,x in enumerate(synopsis_step_one) if not x])
print([i for i,x in enumerate(synopsis_step_two) if not x])
print([i for i,x in enumerate(synopsis_step_three) if not x])

# Many of the movies do not have a synopsis text, this is it won't be used for furhter analysis (althpugh will be extracted)

In [ ]:
"""
Field 6: Extract movie reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_eight)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
"""
Print the movies with no user reviews text.
"""
print([i for i,x in enumerate(myfield_review_step_one) if not x])
print([i for i,x in enumerate(myfield_review_step_two) if not x])
print([i for i,x in enumerate(myfield_review_step_three) if not x])

<b> - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - </b>

#### Dataframe creation based on the movie content, synopsis and reviews extracted

In [ ]:
"""
Import the movie_content (the extracted HTML document of the column imdb_url)
Import the reviews content (the extracted HTML document of the column reviews_url)
"""
with open(os.path.join(os.getcwd(), 'movie_content_url\\data_eight_13012020.pkl'), 'rb') as f:
    
    content_eight = pickle.load(f)

print("Number of URLs: {}".format(len(content_eight)))

with open(os.path.join(os.getcwd(), 'reviews_url\\review_eight_15012020.pkl'), 'rb') as f:
    
    review_eight = pickle.load(f)

print("Number of URLs: {}".format(len(review_eight)))

In [ ]:
"""
Below we import the 5 fields extracted previously
However, we we imported the "rating" file, we noticed that they did not downloaded properly. Actually, the respective list was empty,
thus we decided to download again the data in order no to make any other mistake with the index of the movies.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\pre-indexed files\\plot_eight_18012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\pre-indexed files\\rating_eight_18012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\pre-indexed files\\actors_eight_18012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\pre-indexed files\\director_eight_18012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\pre-indexed files\\synopsis_eight_18012020.pkl'), 'rb') as f:
    
    synopsis = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\pre-indexed files\\reviews_eight_18012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
print(len(plot))
print(len(rating))
print(len(actors))
print(len(director))
print(len(synopsis))
print(len(reviews))

print(rating) #empty

In [ ]:
"""
Running the commands below we can spot the number of indexes that should be removed from the data extracted.
"Add a Plot": Denoted that the movies does have a written plot on its IMDB website.
"It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.": Also denotes a non-written synopsis.

Synopsis indexes are not further considered. There are too many movies with no synopsis text and there is no poin in erasing almost half of the first 5000 movies due to missing synopsis text.
"""
matching_add_plot = [s for s, x in enumerate(plot) if "Add a Plot" in x]

print("Number of movies with 'Add Plot': ", len(matching_add_plot), '\n')

matching_add_synopsis = [s for s, x in enumerate(synopsis) if 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.' in x]

print("Number of movies with no synopsis text: ", len(matching_add_synopsis))

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[44,70,82,94,106,260,263,547,602,603,690,693,694,695,811,906,909,910,939,980,991,994,1124,1236,1276,1469,1526,1625,2160,2165,2181,2190,2235,2241,2298,2317,2347,2423,2511,2603,2766,2772,2793,2879,2884,2885,2886,2891,2933,3053,3054,3139,3152,3181,3192,3196,3213,3384,3387,3393,3538,3630,3818,3828,3927,3964,3974,3977,4027,4186,4248,4308,4342,4371,4504,4568,4628,4657,4740,4756,4787,4828,4861,4862,4863,4864,4865,4866,4867,4868,4869,4871,4873,4904,4932,4933,4935,4936,4942,4949,4968,4973,4975,905,906,907,909,910,1963,3849,4191,906,908,909,4864,4866,4867,906,909,82,122,145,169,316,330,334,438,493,943,945,968,987,996,1073,1086,1254,1274,1276,1308,1315,1352,1354,1388,1562,1614,1641,1653,1654,1661,1954,1958,1960,1967,1990,2029,2030,2031,2192,2347,2383,2391,2488,2553,2558,2565,2572,2576,2607,2817,2833,2855,2893,2963,2969,3012,3044,3080,3154,3211,3225,3426,3434,3555,3591,3611,3614,3634,3745,3749,3821,3823,3834,3837,3856,3885,3992,4003,4009,4020,4021,4094,4127,4147,4150,4188,4200,4240,4244,4314,4387,4447,4515,4530,4591,4593,4599,4601,4667,4732,4762,4830,4875,4882,4917,4919,4938,4966,4970,4972,4974,4978,4988,4990]
index_remove = list(dict.fromkeys(index_remove))
print(len(index_remove))
content_index_eight = [i for j, i in enumerate(content_eight) if j not in index_remove]
review_index_eight = [i for j, i in enumerate(review_eight) if j not in index_remove]
print(len(content_index_eight))
print(len(review_index_eight))

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_eight\\content_index_eight_20012020.pkl'), 'wb') as f:
#     pickle.dump(content_index_eight, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movie_eight\\review_index_eight_20012020.pkl'), 'wb') as f:
#     pickle.dump(review_index_eight, f)

In [ ]:
"""
Import the pruned list of movie_content & reviews web pages. The indexes that have been removed did not contain one or more -
of the following fields: plot, actors, director(s), imdb_rating, review(s)

It is important to re-run the process of extracting the information in order to assure that we have extracted the correct information and text for each movie
Otherwise, we might observe mismatches in the fields assigned to each movie title.
"""
content_souplist_eight = []

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\content_index_eight_20012020.pkl'), 'rb') as f:
    
    content_index_eight = pickle.load(f)

print("Number of URLs: {}".format(len(content_index_eight)))

for i in tqdm_notebook(content_index_eight):
    
    content_souplist_eight.append(BeautifulSoup(i.text))

print("Number of souplists: {}".format(len(content_souplist_eight)))

#-----------------------------------------------------------------------------------------------------

with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\review_index_eight_20012020.pkl'), 'rb') as f:
    
    review_eight = pickle.load(f)

print("Number of URLs: {}".format(len(review_eight)))

review_souplist_eight = []

for i in tqdm_notebook(review_eight):
    
    review_souplist_eight.append(BeautifulSoup(i.text))

print("Number of review tags: {}".format(len(review_souplist_eight)))

In [ ]:
"""
Field 1: Extract plot summary
"""
myfield_plot = []
plot_summary = []
index_to_remove_no_plot = []

[myfield_plot.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eight)]

[[[plot_summary.append(y.text) for y in x.find_all('div', {'class':'summary_text'})] for x in i] if len(i) !=0 else index_to_remove_no_plot.append(myfield_plot.index(i)) for i in myfield_plot]
      
print("Length of Plot Summary list: {}".format(len(plot_summary)))
print("Length of the list with Movies that don't have plot summary: {}".format(len(index_to_remove_no_plot)))
if len(index_to_remove_no_plot) == 0:
    print("None of the movie miss plot")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_plot))

In [ ]:
"""
Field 2: Extract actors
"""
myfield_cast = []
phase_two = []
phase_three = []
actors_list = []
index_to_remove_no_actors = []

[myfield_cast.append(i.find_all('table', {'class':'cast_list'})) for i in tqdm_notebook(content_souplist_eight)]
    
r_one = re.compile(".*name")

[[phase_two.append(j.find_all('a', {'href':r_one})) for j in i] for i in myfield_cast]

[phase_three.append(phase_two[i][1::2]) for i in range(len(phase_two))]

[actors_list.append(list(map(lambda x: x.text.strip(' ').replace('\n', ''), actors))) for actors in phase_three]            
    
index_to_remove_no_actors = [i for i,x in enumerate(myfield_cast) if not x]
        
print("Length of Actors list: {}".format(len(actors_list)))
print("Length of the list with Movies that don't have actors: {}".format(len(index_to_remove_no_actors)))
if len(index_to_remove_no_actors) == 0:
    print("None of the movie miss actors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_actors))

In [ ]:
"""
Field 3: Extract director name(s)
"""
myfield_director = []
director_name = []
index_to_remove_no_directors = []

[myfield_director.append(i.find_all('div', {'class':'plot_summary'})) for i in tqdm_notebook(content_souplist_eight)]

r_name = re.compile(".*name")

[[director_name.append(j.find_all('a', {'href':r_name})) for j in i] for i in myfield_director]
    
director_names = [item[0].text for item in director_name]

index_to_remove_no_directors = [i for i,x in enumerate(myfield_director) if not x]

print("Length of Directors list: {}".format(len(director_names)))
print("Length of the list with Movies that don't have directors: {}".format(len(index_to_remove_no_directors)))
if len(index_to_remove_no_directors) == 0:
    print("None of the movie miss directors")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_directors))

In [ ]:
"""
Field 4: Extract imdb movie rating
"""
myfield_rating = []
ratings = []
index_to_remove_no_rating = []

[myfield_rating.append(i.find_all('div', {'class':'ratingValue'})) for i in tqdm_notebook(content_souplist_eight)]

[[[ratings.append(y.text) for y in x.find_all('span', {'itemprop':'ratingValue'})] for x in i] for i in myfield_rating]

index_to_remove_no_rating = [i for i,x in enumerate(myfield_rating) if not x]

print("Length of Ratings list: {}".format(len(ratings)))
print("Length of the list with Movies that are not rated: {}".format(len(index_to_remove_no_rating)))
if len(index_to_remove_no_rating) == 0:
    print("None of the movie miss ratings")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_rating))

In [ ]:
"""
Field 5: Extract movie user reviews
"""
myfield_review_step_one = []
myfield_review_step_two = []
myfield_review_step_three = []

[myfield_review_step_one.append(i.find_all('div', {'class':'lister-list'})) for i in tqdm_notebook(review_souplist_eight)]

[[myfield_review_step_two.append(j.find_all('div', {'class':'text show-more__control'})) for j in i] for i in myfield_review_step_one]

[myfield_review_step_three.append(list(map(lambda x: x.text, reviews))) for reviews in myfield_review_step_two]

index_to_remove_no_review = [i for i,x in enumerate(myfield_review_step_one) if not x]

print("Length of Reviews list: {}".format(len(myfield_review_step_three)))
print("Length of the list with Movies that don't have reviews: {}".format(len(index_to_remove_no_review)))
if len(index_to_remove_no_review) == 0:
    print("None of the movies miss reviews")
else:
    print("Indexes to remove: {}".format(index_to_remove_no_review))

In [ ]:
print(len([i for i,x in enumerate(myfield_review_step_one) if not x]))

print([i for i,x in enumerate(myfield_review_step_two) if not x])

print([i for i,x in enumerate(myfield_review_step_three) if not x])

In [ ]:
"""
Run this cell once! For every re-execution of this notebook, please use the already pickled file.
"""
# # Pickle the requests file for further use!

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\plot_eight_20012020.pkl'), 'wb') as f:
#     pickle.dump(plot_summary, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\actors_eight_20012020.pkl'), 'wb') as f:
#     pickle.dump(actors_list, f)
    
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\director_eight_20012020.pkl'), 'wb') as f:
#     pickle.dump(director_names, f)

# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\reviews_eight_24012020.pkl'), 'wb') as f:
#     pickle.dump(myfield_review_step_eight, f)

In [ ]:
# with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\rating_eight_20012020.pkl'), 'wb') as f:
#     pickle.dump(ratings, f)

In [ ]:
"""
Remove the faulty indexes from the data. As faulty is characterized an index with no data relevant to movie content or user review of a movie or with an error 404 web page.
Indexes collected during the data extraction of the 6 fields and from the above python cell that prints number of movies with no written plot.
"""
index_remove=[44,70,82,94,106,260,263,547,602,603,690,693,694,695,811,906,909,910,939,980,991,994,1124,1236,1276,1469,1526,1625,2160,2165,2181,2190,2235,2241,2298,2317,2347,2423,2511,2603,2766,2772,2793,2879,2884,2885,2886,2891,2933,3053,3054,3139,3152,3181,3192,3196,3213,3384,3387,3393,3538,3630,3818,3828,3927,3964,3974,3977,4027,4186,4248,4308,4342,4371,4504,4568,4628,4657,4740,4756,4787,4828,4861,4862,4863,4864,4865,4866,4867,4868,4869,4871,4873,4904,4932,4933,4935,4936,4942,4949,4968,4973,4975,905,906,907,909,910,1963,3849,4191,906,908,909,4864,4866,4867,906,909,82,122,145,169,316,330,334,438,493,943,945,968,987,996,1073,1086,1254,1274,1276,1308,1315,1352,1354,1388,1562,1614,1641,1653,1654,1661,1954,1958,1960,1967,1990,2029,2030,2031,2192,2347,2383,2391,2488,2553,2558,2565,2572,2576,2607,2817,2833,2855,2893,2963,2969,3012,3044,3080,3154,3211,3225,3426,3434,3555,3591,3611,3614,3634,3745,3749,3821,3823,3834,3837,3856,3885,3992,4003,4009,4020,4021,4094,4127,4147,4150,4188,4200,4240,4244,4314,4387,4447,4515,4530,4591,4593,4599,4601,4667,4732,4762,4830,4875,4882,4917,4919,4938,4966,4970,4972,4974,4978,4988,4990]
index_remove = list(dict.fromkeys(index_remove))
dataset = pd.read_pickle(os.path.join(os.getcwd(), "dataset_58,000_14012020_latest_version.pkl"))
dataset_eight = dataset.iloc[35000:40000].reset_index(drop=True)
dataset_eight = dataset_eight[~dataset_eight.index.isin(index_remove)]
dataset_eight.shape

In [ ]:
"""
Loading the data extracted.
Those are the most important data source across all the notebook. 
The whole data extraction part was executed to end up with the following 6 files per mini-batch.

The examiner can directly import the files below to review the movie content extracted for the eighth mini-batch.
"""
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\plot_eight_20012020.pkl'), 'rb') as f:
    
    plot = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\rating_eight_20012020.pkl'), 'rb') as f:
    
    rating = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\actors_eight_20012020.pkl'), 'rb') as f:
    
    actors = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\director_eight_20012020.pkl'), 'rb') as f:
    
    director = pickle.load(f)
        
with open(os.path.join(os.getcwd(), '58,000 movies\\movies_eight\\reviews_eight_24012020.pkl'), 'rb') as f:
    
    reviews = pickle.load(f)
    
assert len(plot) == len(rating) == len(actors) == len(director) == len(reviews)

In [ ]:
dataset_eight['actors'] = actors
dataset_eight['plot'] = plot
dataset_eight['imdb_rating'] = rating
dataset_eight['director'] = director
dataset_eight['reviews'] = reviews
dataset_eight = dataset_eight.drop(["movieId", "imdbId", "synopsis_url"], axis=1)

In [ ]:
index_remove=[]
index_remove=[54,58,117,122,151,172,186,191,203,214,224,228,238,271,273,300,314,320,377,435,475,478,492,493,508,515,522,523,546,548,553,562,581,584,675,702,730,735,738,739,742,838,853,857,858,869,876,909,914,953,956,961,973,983,984,1033,1037,1049,1052,1056,1062,1064,1068,1072,1082,1100,1130,1180,1182,1183,1184,1214,1227,1283,1304,1306,1369,1405,1417,1423,1439,1441,1464,1467,1491,1507,1513,1536,1563,1564,1567,1568,1578,1588,1590,1636,1691,1726,1727,1732,1734,1741,1756,1768,1770,1773,1780,1790,1801,1802,1804,1805,1806,1807,1809,1826,1835,1857,1859,1873,1879,1895,1897,1899,1901,1905,1906,1917,1929,1936,1950,1964,1965,1968,1971,1976,1979,1981,1988,1994,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2008,2009,2010,2012,2043,2059,2109,2117,2118,2120,2121,2122,2125,2126,2127,2137,2151,2170,2174,2179,2203,2220,2229,2235,2289,2291,2292,2293,2294,2315,2324,2329,2330,2332,2341,2359,2361,2363,2365,2366,2368,2369,2370,2371,2388,2401,2409,2428,2432,2433,2435,2461,2462,2463,2466,2468,2471,2472,2476,2477,2482,2484,2485,2488,2490,2491,2492,2494,2496,2499,2501,2504,2507,2521,2626,2641,2656,2667,2670,2702,2705,2750,2762,2803,2805,2817,2829,2867,2889,2907,2915,2951,2962,2966,2973,2975,3030,3045,3047,3048,3064,3065,3078,3090,3095,3096,3101,3109,3119,3148,3162,3174,3179,3188,3189,3197,3198,3206,3210,3211,3215,3216,3217,3218,3221,3222,3223,3224,3226,3294,3299,3305,3320,3323,3324,3329,3340,3345,3361,3371,3381,3383,3385,3388,3398,3399,3404,3409,3432,3433,3439,3444,3446,3467,3470,3475,3484,3485,3489,3506,3536,3537,3546,3557,3562,3567,3575,3580,3582,3584,3588,3589,3595,3596,3607,3610,3615,3616,3645,3649,3674,3681,3688,3690,3704,3718,3730,3742,3746,3762,3768,3770,3777,3789,3790,3794,3804,3809,3810,3826,3858,3863,3866,3867,3868,3873,3949,3966,3967,3975,3997,3999,4001,4021,4029,4050,4060,4064,4069,4075,4076,4085,4086,4089,4110,4129,4143,4171,4173,4176,4204,4208,4210,4214,4227,4229,4243,4246,4265,4288,4289,4341,4383,4394,4396,4421,4422,4424,4425,4429,4489,4497,4510,4521,4537,4542,4581,4595,4625,4642,4676,4713,4717,4726,4733,4760,4764]
dataset_eight = dataset_eight[~dataset_eight.index.isin(index_remove)]
dataset_eight = dataset_eight[dataset_eight.astype(str)['reviews'] != '[]']
dataset_eight.shape

In [ ]:
"""
This is the table to use in Part 1, and the table that will be concatenated with the rest of the mini-batch tables.
"""
# dataset_eight.to_pickle(os.path.join(os.getcwd(), "58,000 movies\\movie_eight\\dataset_eight_final_25012020.pkl"))